In [2]:
import pandas as pd
import numpy  as np

Task 1: Create Import file (try, except)

In [5]:
name = input()
try:
    with open(name+'.txt','r') as gs:
        grades = gs.readlines()
except IOError:
        print("File not accessible")
        

File not accessible


Task 2: Inspect Value (check if has 26 values, print invalid, report)

for loop to check if has 26 values, if not, print

In [29]:
checklist = grades

for grade in grades: 
            value = grade.split(",")
            length = len(value)
            print(grade)


N00000001,A,A,D,D,C,D,D,A,,C,D,B,C,,B,C,B,D,A,C,,A,,C,D

N00000002,,A,,D,,B,D,A,C,C,D,,A,A,A,C,B,D,C,C,A,A,B,,D

N00000003,B,A,,D,C,B,D,A,C,C,,B,A,B,A,C,B,D,A,,A,,B,D,D

N00000004,B,B,D,,,B,D,A,C,C,D,B,A,B,A,C,B,D,,C,A,D,B,C,D

N00000005,B,A,,D,,B,D,A,C,C,D,B,A,B,A,C,B,D,D,C,A,A,,D,D

N00000006,B,A,D,A,C,B,D,A,C,C,C,B,A,D,A,C,,C,A,C,A,C,B,D,A

N00000007,B,A,D,D,C,B,D,A,C,,D,B,A,B,A,C,B,D,,C,A,A,B,D,

N00000008,A,A,D,D,C,,D,A,C,C,B,C,A,B,A,A,B,D,A,C,A,,B,B,D

N00000009,B,A,,D,C,B,D,A,C,A,D,B,A,B,A,A,B,D,A,C,A,A,B,D,D

N00000010,,A,D,B,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,D,,,B,D,B

N00000011,B,A,D,,C,B,D,A,C,D,D,B,A,B,A,C,B,D,A,C,A,B,B,D,D

N00000012,B,,D,D,C,B,D,A,C,C,D,B,A,B,A,A,B,D,A,,A,A,A,,C

N00000013,,A,D,,B,B,,A,C,A,D,B,A,B,D,C,B,D,A,C,A,A,B,D,D

N00000014,B,B,D,D,C,B,,D,C,C,D,B,A,B,A,C,,D,A,C,A,A,,D,D

N00000015,A,C,C,D,C,B,,A,C,C,D,B,A,B,A,C,B,D,A,C,A,B,,D,D

N00000016,B,A,C,D,,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D

N00000017,B,A,D,D,C,B,C,A,D,C,D,A,A,D,A,C,B,D,D,B,,A,B,D,D

N00000

Task 3: Calculate Exam Test:
How many high score 
average score
highest score
lowest score 
range of score(highest minus lowest) 
question most people skip(question num - how many)
question most answer incorrectly(question num - how many)

----------
>>> from collections import Counter
>>> z = ['blue', 'red', 'blue', 'yellow', 'blue', 'red']
>>> Counter(z)
Counter({'blue': 3, 'red': 2, 'yellow': 1})

Task 4: Export text file: 
as xxx_grades.txt
list as 'id', 'score'